## 5. Evaluate the model using TFMA

In [ ]:
import tensorflow_model_analysis as tfma

### 5.1 Evaluate input function

In [ ]:
def generate_eval_receiver_fn(transform_artefacts_dir):
    
    transformed_metadata = metadata_io.read_metadata(transform_artefacts_dir+"/transformed_metadata")
    transformed_feature_spec = transformed_metadata.schema.as_feature_spec()
    
    def _eval_receiver_fn():
        
        serialized_tf_example = tf.placeholder(
            dtype=tf.string, shape=[None], name='input_example_placeholder')

        receiver_tensors = {'examples': serialized_tf_example}
        transformed_features = tf.parse_example(serialized_tf_example, transformed_feature_spec)

        return tfma.export.EvalInputReceiver(
            features=transformed_features,
            receiver_tensors=receiver_tensors,
            labels=transformed_features[TARGET_FEATURE_NAME])

    return _eval_receiver_fn

### 5.2 Export Evaluation Saved Model

In [ ]:
eval_model_dir = model_dir +"/export/evaluate"

transform_artefacts_dir = os.path.join(local_models_dir,TRANSFORM_ARTEFACTS_DIR)

tfma.export.export_eval_savedmodel(
        estimator=dnn_estimator,
        export_dir_base=eval_model_dir,
        eval_input_receiver_fn=generate_eval_receiver_fn(transform_artefacts_dir)
)

### 5.3 Produce Evaluation Results using the Saved Model

In [ ]:
slice_spec = [tfma.SingleSliceSpec()]
for feature_name, feature_spec in transformed_feature_spec.items():
    if feature_name not in [KEY_COLUMN] + [TARGET_FEATURE_NAME] and feature_spec.dtype == tf.string:
        slice_spec += [tfma.SingleSliceSpec(columns=[feature_name])]

print slice_spec
print ""

saved_model_base_dir=os.path.join(model_dir,'export/evaluate')
model_location=os.path.join(saved_model_base_dir, os.listdir(saved_model_base_dir)[0])
data_location = os.path.join(local_data_dir, TRANSFORMED_DATA_DIR)+"/eval-*.tfrecords"

tf.logging.set_verbosity(tf.logging.ERROR)

eval_results = tfma.run_model_analysis(
    model_location=model_location , 
    data_location=data_location, 
    file_format='tfrecords', 
    slice_spec=slice_spec, 
#     output_path=None
)

print "Evaluation results are ready!"

### 5.4 Visualise the Results

In [ ]:
print eval_results.slicing_metrics

In [ ]:
tfma.view.render_slicing_metrics(
        eval_results, 
    slicing_column='mother_race'
)